In [31]:
import os
import json
import numpy as np
import pandas as pd


In [32]:
# WRITE PATH TO DATA DIRECTORY BELOW 

data_path = '../data/redclassic/'

files = []
data = {}
files_received = []
load_files = []
driver_files = []

try:
    for i in os.listdir(data_path):
        if os.path.isfile(os.path.join(data_path,i)):
            files.append(i)
except FileNotFoundError:
    print("PLEASE MAKE SURE THE DATA PATH LEADS TO THE DATA DIRECTORY WHERE THE DATA FILES ARE LOCATED\n")   
except NameError:
    print("PLEASE INSERT A VALID PATH STRING TO THE DATA DIRECTORY\n")
    
if '.DS_Store' in files:
    files.remove('.DS_Store')
for file in files:
    data[file] = pd.read_csv(data_path + file, low_memory=False)

for file in files:
    try:
        data[file]
        files_received.append(file)
        if 'driver' in file.lower():
            driver_files.append(file)
        else:
            load_files.append(file)
    except KeyError:
        print(f'** The {file} file is missing from the data folder **'.upper())        

if len(driver_files) == 1:
    print(f"Received {len(driver_files)} driver file")
else:
    print(f"Received {len(driver_files)} driver files")
    
if len(load_files) == 1:
    print(f"Received {len(load_files)} load file")
else:
    print(f"Received {len(load_files)} load files")

if len(files_received) == 0:
    print("\n!!! NO FILES RECEIVED !!!")
    
for file in files_received:
    print(f"{file} - {type(data[file])}")

Received 1 driver file
Received 4 load files
OD Updated RCT Driver File.csv - <class 'pandas.core.frame.DataFrame'>
OPT DYNAMICS 2021 Q2.csv - <class 'pandas.core.frame.DataFrame'>
OPT DYNAMICS 2021 Q3.csv - <class 'pandas.core.frame.DataFrame'>
OPT DYNAMICS 2021 Q4.csv - <class 'pandas.core.frame.DataFrame'>
RED_CLASSIC OPT DYNAMICS 2021 Q1.csv - <class 'pandas.core.frame.DataFrame'>


In [33]:
# READ IN ANY CONFIGURATION FILES
config_path = '../config_files/redclassic/'
config_files = []
config_files_received = []
config_json = {}

try:
    for i in os.listdir(config_path):
        if os.path.isfile(os.path.join(config_path,i)):
            config_files.append(i)
except FileNotFoundError:
    print("PLEASE MAKE SURE THE CONFIG_FILE PATH LEADS TO THE CONFIG DIRECTORY WHERE THE CONFIG FILES ARE LOCATED\n")   
except NameError:
    print("PLEASE INSERT A VALID PATH STRING TO THE CONFIG FILE DIRECTORY\n")

for file in files:
    data[file] = pd.read(data_path + file, low_memory=False)
    
for file in config_files:
    try:
        config_json[file]
        config_files_received.append(file)
    except KeyError:
        print(f'** The {file} file is missing from the config folder **'.upper())
    
    
for file in config_files_received:
    print(f"{file} - {type(config_json[file])}")

AttributeError: module 'pandas' has no attribute 'read'

In [34]:
# Combine all files of similar type to one file
load_data_combined_list = [data[file] for file in load_files]
    
load_data_combined_df = pd.concat(load_data_combined_list, ignore_index=True)

print(load_data_combined_df.info())
print(load_data_combined_df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1261246 entries, 0 to 1261245
Data columns (total 36 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   MOV_TYPE              1261246 non-null  object 
 1   MOV_EFFDATE           1261246 non-null  object 
 2   ORD_HDRNUMBER         1227189 non-null  float64
 3   ORD_BILLTO            1227189 non-null  object 
 4   ORD_ORIGIN_ZIP        1227186 non-null  object 
 5   ORD_DEST_ZIP          1227189 non-null  object 
 6   ORD_BOOKDATE          1227189 non-null  object 
 7   ORD_STARTDATE         1227189 non-null  object 
 8   ORD_COMPLETIONDATE    1227189 non-null  object 
 9   REV_TOT               1226514 non-null  float64
 10  REV_LHF               1226514 non-null  float64
 11  REV_FSC               1226514 non-null  float64
 12  REV_ACC               1226514 non-null  float64
 13  LGH_NUMBER            1261246 non-null  int64  
 14  LGH_CARRIER           1261246 non-

In [ ]:
# Run Load Files Checks

# LIST THE COLUMNS REQUIRED (NO BLANKS)
required_load_columns = []

for file in load_files:
    df = data[file]
    print(df.info())
    print(df.isnull().sum())

In [36]:
# Run Driver Files Checks
blanks = {}

required_driver_columns = list(data[driver_files[0]].columns.values)

# Columns to remove from required list
columns_not_required = required_driver_columns[7:]

for file in driver_files:
    df = data[file]
    print(df.info())
    
#     Find Blanks for required columns
    for column in required_driver_columns:
        if column in columns_not_required:
            pass
        else:
            column_blanks_df = df[df[column].isnull()]
            blanks[column] = column_blanks_df

for column in blanks.keys():
    if blanks[column].empty == False:
        print(blanks[column])
    else:
        pass

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1099 entries, 0 to 1098
Data columns (total 10 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Driver ID                              1099 non-null   object 
 1   Driver First Name                      1099 non-null   object 
 2   Driver Last Name                       1099 non-null   object 
 3   Home or Home Terminal Zip Code         1099 non-null   object 
 4   Home City                              1097 non-null   object 
 5   Home State                             1099 non-null   object 
 6   Driver Type (as listed in parameters)  1099 non-null   object 
 7   Permits                                0 non-null      float64
 8   Subnetwork ID (if applicable)          0 non-null      float64
 9   Current Equipment Type                 0 non-null      float64
dtypes: float64(3), object(7)
memory usage: 86.0+ KB
None
    Driver ID Drive

In [42]:
series = load_data_combined_df['STP_TYPE'] + '-' + load_data_combined_df['STP_EVENT_DESC'] + '-' + load_data_combined_df['STP_LOADSTATUS_GROUP']
print(series.unique())
print(series.value_counts())

['NONE-Begin Empty-Empty' 'NONE-End Empty-Empty'
 'NONE-Drop Empty Trailer-Empty' 'NONE-End Bobtail-Empty'
 'PUP-Live Load-Empty' 'DRP-Final Drop Loaded Tr-Loaded'
 'NONE-Drop Loaded Trailer-Loaded' 'NONE-Hook Empty Trailer-Empty'
 'NONE-Hook Loaded Trailer-Empty' 'DRP-Live Unload-Loaded'
 'PUP-Live Load-Loaded' 'PUP-Hook Preloaded Trl-Empty'
 'NONE-Route Point-Empty' 'NONE-Route Point-Loaded'
 'PUP-Hook Preloaded Trl-Loaded' 'NONE-Begin Bobtail-Empty' nan
 'NONE-Track Point-Empty' 'NONE-Placed In Service-Empty'
 'NONE-Begin Empty-Loaded' 'NONE-Drop Empty Trailer-Loaded'
 'UNK-Placed In Service-Empty' 'NONE-Billable Customs-Loaded'
 'NONE-Billable Customs-Empty' 'NONE-Begin Bobtail-Loaded'
 'NONE-Change Trailer(s)-Empty' 'NONE-Chg Trl at Hook Loc-Empty']
DRP-Live Unload-Loaded             320121
NONE-Begin Empty-Empty             188944
PUP-Live Load-Loaded               165553
NONE-Drop Empty Trailer-Empty      114635
PUP-Live Load-Empty                100836
PUP-Hook Preloaded Trl-Em